<a href="https://colab.research.google.com/github/davidclizbe/datascience/blob/master/Clizbe_Time_Series_Day_3_Morning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Decomposition & Analysis Assignment

In [ ]:
import pandas as pd
import statsmodels.api as sm
import plotly.express as px

### Import the monthly sunspots data set into a Pandas dataframe, and convert the Month field to a datetime data type.

In [ ]:
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Time%20Series%20Analysis/Time%20Series%20-%20Day%203/monthly-sunspots.csv')
df

,Month,Sunspots
0,1749-01,58.0
1,1749-02,62.6
2,1749-03,70.0
3,1749-04,55.7
4,1749-05,85.0
...,...,...
2815,1983-08,71.8
2816,1983-09,50.3
2817,1983-10,55.8
2818,1983-11,33.3


### Use a line chart to plot the time series.

### Decompose the time series and add columns for the trend, seasonality, and residuals to the data set.

In [ ]:
series = df.set_index('Month')
decomposition = sm.tsa.seasonal_decompose(series, model= 'additive', freq= 365)

trend = decomposition.trend
trend = trend[trend['Sunspots'].isna() == False].reset_index()

In [ ]:
seasonality = decomposition.seasonal.reset_index()

seasonality

,Month,Sunspots
0,1749-01,3.901250
1,1749-02,9.539880
2,1749-03,4.195027
3,1749-04,2.799488
4,1749-05,-0.989553
...,...,...
2815,1983-08,1.404420
2816,1983-09,2.536044
2817,1983-10,3.828256
2818,1983-11,0.849586


In [ ]:
residuals = decomposition.resid.reset_index()
residuals

,Month,Sunspots
0,1749-01,NaN
1,1749-02,NaN
2,1749-03,NaN
3,1749-04,NaN
4,1749-05,NaN
...,...,...
2815,1983-08,NaN
2816,1983-09,NaN
2817,1983-10,NaN
2818,1983-11,NaN


In [ ]:
df_merged = df.merge(trend, on= 'Month')
df_merged.columns = ['Month', 'Observed', 'Trend']
df_merged = df_merged.merge(seasonality, on= 'Month')
df_merged.columns = ['Month', 'Observed', 'Trend', 'Seasonality']
df_merged

,Month,Observed,Trend,Seasonality
0,1764-03,40.2,53.876438,-7.991118
1,1764-04,34.4,54.029041,-12.284073
2,1764-05,44.3,54.249315,-3.731862
3,1764-06,30.0,54.364384,-6.209866
4,1764-07,30.0,54.515890,-4.434484
...,...,...,...,...
2451,1968-06,110.3,79.445479,5.879684
2452,1968-07,96.1,79.507123,13.771974
2453,1968-08,109.3,79.625753,10.516162
2454,1968-09,117.2,79.657260,9.443168


In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2456 entries, 0 to 2455
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Month        2456 non-null   object 
 1   Observed     2456 non-null   float64
 2   Trend        2456 non-null   float64
 3   Seasonality  2456 non-null   float64
dtypes: float64(3), object(1)
memory usage: 95.9+ KB


In [ ]:
px.line(df_merged, x='Month', y= 'Observed', title= 'Sunspots vs Time (months)').update(layout=dict(title=dict(x=0.5)))


In [ ]:
px.line(df_merged, x='Month', y= 'Trend', title= 'Sunspots vs Time (months)').update(layout=dict(title=dict(x=0.5)))

In [ ]:
px.line(df_merged, x='Month', y= 'Seasonality', title= 'Sunspots vs Time (months)').update(layout=dict(title=dict(x=0.5)))

### Plot the observed values, trend, seasonality, and residuals on a multi-line chart. You should have a line for each column.

**Hint:** You may need to melt the data so that all the categories are in a single column and all the values are in a single column.

In [ ]:
ids =  ['Month']
melt_fields = ['Observed','Trend', 'Seasonality']
melted_data = pd.melt(df_merged, id_vars = ids , value_vars = melt_fields)
melted_data

,Month,variable,value
0,1764-03,Observed,40.200000
1,1764-04,Observed,34.400000
2,1764-05,Observed,44.300000
3,1764-06,Observed,30.000000
4,1764-07,Observed,30.000000
...,...,...,...
7363,1968-06,Seasonality,5.879684
7364,1968-07,Seasonality,13.771974
7365,1968-08,Seasonality,10.516162
7366,1968-09,Seasonality,9.443168


In [ ]:
px.line(melted_data, x='Month', y= 'value',color= 'variable', title= 'Sunspots vs Time (months)').update(layout=dict(title=dict(x=0.5)))

In [ ]:
melted_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7368 entries, 0 to 7367
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Month     7368 non-null   object 
 1   Month     7368 non-null   object 
 2   Sunspots  7368 non-null   float64
dtypes: float64(1), object(2)
memory usage: 172.8+ KB


### Add two columns to the data set - one that calculates a rolling mean and another that calculates a rolling standard deviation.

### Plot the rolling mean and standard deviation on a multi-line chart along with the observed values.

### Perform an Augmented Dickey Fuller Test on the data set and determine whether this time series is stationary.

### Generate an autocorrelation plot for the data set to determine the level of autocorrelation in this time series.